In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 15, 12

from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm

# import seaborn as sns

# from imblearn.over_sampling import SMOTE

In [16]:
df = pd.read_csv("/Users/ianchang/0.Workspaces/slaCoefficientProject/Origination Data/DTT SLA Summary - Overall Summary.csv").drop(columns=["Unnamed: 0", "partner_id", "partner", "xfr_request_id", "request_id", "true_SLA", "runtime_per_day", "payment_code"])

In [17]:
df['submit_date'] = pd.to_datetime(df['submit_date'])
df["submit_date"] = df["submit_date"].dt.floor('D')

# Extract year and month, then create the submit_month column
df['submit_month'] = df['submit_date'].dt.to_period('M')

In [18]:
origination_by_month = df[["on_time", "submit_month"]].groupby(by="submit_month").mean().reset_index()


In [30]:
# fig, ax = plt.subplots(figsize = (10,5))
# plt.style.use('bmh')

# ax.plot(origination_by_month["submit_month"].astype("str"), origination_by_month["on_time"], marker="v", label="Monthly OTD%", color="blue")


# ax.set_ylim(0.9, 1)
# ax.axhline(y=0.95, color='red', linestyle='--', label="Desired Level of OTD%")
# ax.set_title("Monthly Trend of On Time Delivery Percentage (OTD%)")
# ax.set_xlabel("Month")
# ax.set_ylabel("% in Decimals")

# plt.xticks(rotation=45)
# plt.legend()
# plt.show()

In [24]:
import random
import string

def get_random_string(length=8):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

df['author'] = df['author'].str.replace(r'@.*', '', regex=True)

unique_authors = df['author'].unique()

author_mapping = {}

for author in unique_authors:
    author_mapping[author] = get_random_string()

df['author_anonymous'] = df['author'].map(author_mapping)



In [55]:
df = df.sort_values(by="author_anonymous")
df[df["asset_type"] == "LANGUAGE_TEMPLATE"]

,author_anonymous,title_id,package_id,title_desc,asset_type,content_category,reason_code,original_audio_language,source_language,target_language,runtime_minutes,current_state,submit_date,submit_month,due_date,start_date,on_time,billing_events
10818,AWwdBLro,81566176,2101823,A Night at the Kindergarten,LANGUAGE_TEMPLATE,VIDEO,STANDARD,pl,--,pl,93.4,AUTHORING_COMPLETE,2022-10-17,2022-10,2022-10-18,2022-10-10,1,NaN
8906,AWwdBLro,81564732,2047778,Broad Peak,LANGUAGE_TEMPLATE,VIDEO,STANDARD,pl,--,pl,92.5,AUTHORING_COMPLETE,2022-06-29,2022-06,2022-06-30,2022-06-21,1,NaN
928,ETpUnCgd,81450531,2136552,"Jewish Matchmaking: Season 1: ""Episode 6""",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,25.5,AUTHORING_COMPLETE,2023-01-28,2023-01,2023-01-28,2023-01-26,1,NaN
367,ETpUnCgd,81450527,2136540,"Jewish Matchmaking: Season 1: ""Episode 2""",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,39.1,AUTHORING_COMPLETE,2023-01-15,2023-01,2023-01-18,2023-01-06,1,NaN
1038,ETpUnCgd,81450532,2136551,"Jewish Matchmaking: Season 1: ""Episode 7 So Th...",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,31.2,AUTHORING_COMPLETE,2023-01-31,2023-01,2023-02-01,2023-01-28,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4936,xLLXCvDP,81571934,2026632,Trees of Peace,LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,97.1,AUTHORING_COMPLETE,2022-02-13,2022-02,2022-02-13,2022-02-03,1,NaN
6390,xLLXCvDP,81487582,1997312,"Riverdale: Season 6: ""Chapter One Hundred and ...",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,42.1,AUTHORING_COMPLETE,2022-04-15,2022-04,2022-04-16,2022-04-10,1,NaN
358,xLLXCvDP,81597267,2096066,"Never Have I Ever: Season 4: ""Episode 9""",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,29.4,AUTHORING_COMPLETE,2023-01-14,2023-01,2023-01-15,2023-01-10,1,NaN
9750,xLLXCvDP,81553907,2013204,"Riverdale: Season 6: ""Chapter One Hundred and ...",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,42.1,AUTHORING_COMPLETE,2022-07-26,2022-07,2022-07-26,2022-07-22,1,NaN


In [56]:
# df.groupby(by=["author_anonymous", "asset_type", "reason_code", "source_language", "target_language"]).agg({"runtime_minutes": "sum"}).head(50)

In [69]:
# Initialize  
df["expected_daily_output"] = 0

for index, row in df.iterrows():
    if row["asset_type"] == "SUBTITLE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

    elif row["asset_type"] == "CLOSED_CAPTIONING":
        if row["original_audio_language"] == row["target_language"]:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 30
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 60
        else:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 15
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 30
    
    elif row["asset_type"] == "LANGUAGE_TEMPLATE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

df = df[['author_anonymous','title_id', 'package_id', 'title_desc', 'asset_type',
       'content_category', 'reason_code', 'original_audio_language',
       'source_language', 'target_language',"expected_daily_output", 'runtime_minutes',
        'start_date', 'submit_date', 'due_date', 'on_time',
       'billing_events']]

In [70]:
df

,author_anonymous,title_id,package_id,title_desc,asset_type,content_category,reason_code,original_audio_language,source_language,target_language,expected_daily_output,runtime_minutes,start_date,submit_date,due_date,on_time,billing_events
6704,AWwdBLro,81293415,2006906,"Endless Night: Season 1: ""Episode 2""",SUBTITLE,VIDEO,STANDARD,fr,en,pl,15,29.5,2022-04-21,2022-04-27,2022-04-27,1,357.0
11536,AWwdBLro,81572885,2074895,"Love Is Blind: Brazil: Season 2: ""Ep 9 - No Ch...",SUBTITLE,VIDEO,STANDARD,pt-BR,en,pl,15,53.2,2022-11-07,2022-11-11,2022-11-12,1,953.0
4463,AWwdBLro,81262432,1973826,"Human Resources: Season 1: ""Bad Mummies""",SUBTITLE,VIDEO,STANDARD,en,en,pl,15,26.7,2022-01-14,2022-01-18,2022-01-19,1,479.0
8161,AWwdBLro,81228848,2011968,"Ridley Jones: Season 4: ""Escape Tomb / Dante t...",SUBTITLE,VIDEO,STANDARD,en,en,pl,15,24.6,2022-06-06,2022-06-07,2022-06-08,1,466.0
6054,AWwdBLro,81229488,2020835,"FROM SCRATCH: Season 1: ""Between the Fire and ...",SUBTITLE,VIDEO,STANDARD,en,en,pl,15,56.4,2022-03-29,2022-04-04,2022-04-05,1,614.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9653,zxRjUojl,81446385,2061441,"The Green Glove Gang: Season 1: ""Episode 3""",SUBTITLE,VIDEO,STANDARD,pl,en,ko,15,32.3,2022-07-21,2022-07-22,2022-07-22,1,270.0
8153,zxRjUojl,81506540,2036833,"Never Have I Ever: Season 3: ""Episode 4""",SUBTITLE,VIDEO,STANDARD,en,en,ko,15,25.6,2022-06-02,2022-06-07,2022-06-07,1,493.0
7697,zxRjUojl,81447086,2054195,Inside the Mind of a Cat,SUBTITLE,VIDEO,STANDARD,en,en,ko,15,66.7,2022-05-20,2022-05-26,2022-05-27,1,949.0
9991,zxRjUojl,81446386,2061443,"The Green Glove Gang: Season 1: ""Episode 4""",SUBTITLE,VIDEO,STANDARD,pl,en,ko,15,34.6,2022-08-01,2022-08-02,2022-08-02,1,405.0
